# Segmenting and Clustering Neighborhoods in Toronto

## Part 1 - Scraping wiki page and get the lists of neighborhood in Toronto 

In [ ]:
!conda install -c anaconda beautifulsoup4

In [3]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: ...working... done

# All requested packages already installed.



In [4]:
!pip install geocoder

In [28]:
#import packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import geocoder 
import folium

In [7]:
# requests data from wiki ppage and create the dataframe
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r=requests.get(url)
#print(r)
soup=BeautifulSoup(r.text,'html.parser')
#print(soup)

table=soup.find_all('table')[0]
#print(table)
all_postalcode=table.find_all('p')
#print(len(all_postalcode))
data=[]

PostalCode=[]
Borough=[]
Neighborhood=[]
for p in all_postalcode:
    
    PCode=p.find('b').text
    #print(PCode)
    Borough_i=p.text.replace(PCode,'')
    #print(Borough_i)
    if Borough_i=="Not assigned\n":
        pass
    else:
        PostalCode=PCode
        Borough_Nei=Borough_i.split('(')
        #print(Borough_Nei)
        Borough=Borough_Nei[0]
        #print(Borough)
        Neighborhood=Borough_Nei[1].replace(')\n','')
        Neighborhood=Neighborhood.replace('\n','')
        Neighborhood=Neighborhood.split(')')[0]
        #print(PostalCode, Borough, Neighborhood)      
        data.append([PostalCode,Borough,Neighborhood])
#print(data)
df=pd.DataFrame(data,columns=['PostalCode','Borough','Neighborhood'])
#print(df[df['PostalCode']=='M3N'])
df=df.drop_duplicates()
print(df.shape)
df

(103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


## Part 2 - get the latitude and longitude of all the neighborhoods

get the latitude and longitude of each borough of Toronto

In [15]:
borough_list=df['Borough'].values

Latitude=[]
Longitude=[]


for borough_each in borough_list:
    #print(borough_each)
    try:
        geolocator = Nominatim(user_agent="toronto_explorer")

        g=geolocator.geocode('{}, Toronto, Ontario'.format(borough_each))
        #print("g is {}".format(g) )
      
                      
        Latitude.append(g.latitude)
        Longitude.append(g.longitude)
    except:
        Latitude.append(0)
        Longitude.append(0)
    
        
#print(Latitude,Longitude)
df_complete=df.copy()

df_complete['Latitude']=Latitude
df_complete['Longitude']=Longitude
df_complete

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.754326,-79.449117
1,M4A,North York,Victoria Village,43.754326,-79.449117
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.656322,-79.380916
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.754326,-79.449117
4,M7A,Queen's Park,Ontario Provincial Government,43.659659,-79.390340
...,...,...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.643556,-79.565633
99,M4Y,Downtown Toronto,Church and Wellesley,43.656322,-79.380916
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,0.000000,0.000000
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.643556,-79.565633


clean the data, remove one borough that has the wrong latitude and longitude (which shows 0)

In [17]:
df_complete[df_complete['Latitude']==0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
35,M4J,East YorkEast Toronto,The Danforth East,0.0,0.0
76,M7R,MississaugaCanada Post Gateway Processing Centre,Enclave of L4W,0.0,0.0
92,M5W,Downtown TorontoStn A PO Boxes25 The Esplanade,Enclave of M5E,0.0,0.0
94,M9W,EtobicokeNorthwest,Clairville / Humberwood / Woodbine Downs / Wes...,0.0,0.0
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,0.0,0.0


In [19]:
df_complete=df_complete[df_complete['Latitude']!=0]
df_complete.shape

(98, 5)

explode each neighborhood a row

In [23]:
df_new_1=df_complete.copy()
df_new_1['Neighborhood']=df_new_1['Neighborhood'].str.split('/')
df_final=df_new_1.explode('Neighborhood').reset_index(drop=True)
# replace the wrong name in the dataframe
df_final['Neighborhood']=df_final['Neighborhood'].str.replace('Beaumond Heights ' , 'Beaumonde Heights')
df_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.754326,-79.449117
1,M4A,North York,Victoria Village,43.754326,-79.449117
2,M5A,Downtown Toronto,Regent Park,43.656322,-79.380916
3,M5A,Downtown Toronto,Harbourfront,43.656322,-79.380916
4,M6A,North York,Lawrence Manor,43.754326,-79.449117
...,...,...,...,...,...
197,M8Z,Etobicoke,Mimico NW,43.643556,-79.565633
198,M8Z,Etobicoke,The Queensway West,43.643556,-79.565633
199,M8Z,Etobicoke,South of Bloor,43.643556,-79.565633
200,M8Z,Etobicoke,Kingsway Park South West,43.643556,-79.565633


get the latitude and longitude of each neighborhood

In [24]:
neighborhood_list=df_final['Neighborhood'].values
Latitude=[]
Longitude=[] 
for each_neighborhood in neighborhood_list:
    #print(each_neighborhood)
    try:
        geolocator = Nominatim(user_agent="toronto_explorer")
        g=geolocator.geocode('{}, Toronto, Ontario'.format(each_neighborhood))
        #print("g is {}".format(g) )
                     
        Latitude.append(g.latitude)
        Longitude.append(g.longitude)
    except:
        Latitude.append(0)
        Longitude.append(0)
        
#print(Latitude,Longitude)

df_final['Latitude']=Latitude
df_final['Longitude']=Longitude
df_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
3,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
4,M6A,North York,Lawrence Manor,43.722079,-79.437507
...,...,...,...,...,...
197,M8Z,Etobicoke,Mimico NW,43.616677,-79.496805
198,M8Z,Etobicoke,The Queensway West,43.623618,-79.514764
199,M8Z,Etobicoke,South of Bloor,43.667662,-79.394698
200,M8Z,Etobicoke,Kingsway Park South West,43.647381,-79.511333


clean the data, remove the neighborhood with incorrect lat and lng

In [25]:
df_final[df_final['Latitude']==0]


,PostalCode,Borough,Neighborhood,Latitude,Longitude
6,M7A,Queen's Park,Ontario Provincial Government,0.0,0.0
13,M5B,Downtown Toronto,"Garden District, Ryerson",0.0,0.0
36,M6E,York,Caledonia-Fairbanks,0.0,0.0
100,M6M,York,Keelsdale and Silverthorn,0.0,0.0
166,M8V,Etobicoke,Humber Bay Shores,0.0,0.0


In [26]:
df_final=df_final[df_final['Latitude']!=0]
df_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
3,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
4,M6A,North York,Lawrence Manor,43.722079,-79.437507
...,...,...,...,...,...
197,M8Z,Etobicoke,Mimico NW,43.616677,-79.496805
198,M8Z,Etobicoke,The Queensway West,43.623618,-79.514764
199,M8Z,Etobicoke,South of Bloor,43.667662,-79.394698
200,M8Z,Etobicoke,Kingsway Park South West,43.647381,-79.511333


## Part 3 - explore and cluster the neighborhoods in Toronto

### 3.1 Create a map of Toronto with neighborhoods superimposed on top

In [29]:
#Use geopy library to get the latitude and longitude values of Toronto.
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto



The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### 3.2 Use Foursquare API to get the venues in each neighborhood in Toronto

#### 3.2.1 Define Foursquare Credentials and Version

In [30]:
CLIENT_ID = 'OQMBZOQH4UD5GJCMBNF1T1AIYWIXWAT1BKFFMGBGKOBS1KJU' # your Foursquare ID
CLIENT_SECRET = 'YFMPLQAXMVGUER1HN2GJ0W23MOKM3Z5TKUOS4TQFJ2E2KZXD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OQMBZOQH4UD5GJCMBNF1T1AIYWIXWAT1BKFFMGBGKOBS1KJU
CLIENT_SECRET:YFMPLQAXMVGUER1HN2GJ0W23MOKM3Z5TKUOS4TQFJ2E2KZXD


#### 3.2.2 try 1 neighborhood with foursquare api, get the names and categories of nearby venues

In [34]:
neighborhood_1=df_final.loc[0,'Neighborhood']
latitude_1=df_final.loc[0,'Latitude']
longitude_1=df_final.loc[0,'Longitude']
print('Latitude and longitude of {} is {},{}'.format(neighborhood_1,latitude_1,longitude_1,))

limit=100
radius=500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
     CLIENT_ID,
     CLIENT_SECRET,
     VERSION,
     latitude_1,
     longitude_1,
     radius,
     limit)
print(url)

import requests
from pandas.io.json import json_normalize
results=requests.get(url).json()
#print(len(results))
#print(results['response']['groups'][0])
venues=results['response']['groups'][0]['items']
#print(type(venues))
#print(len(venues))
#print(venues[0])
nearby_venues=json_normalize(venues)
#print(nearby_venues)
#print(type(nearby_venues))
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]
nearby_venues

Latitude and longitude of Parkwoods is 43.7587999,-79.3201966
https://api.foursquare.com/v2/venues/explore?&client_id=OQMBZOQH4UD5GJCMBNF1T1AIYWIXWAT1BKFFMGBGKOBS1KJU&client_secret=YFMPLQAXMVGUER1HN2GJ0W23MOKM3Z5TKUOS4TQFJ2E2KZXD&v=20180605&ll=43.7587999,-79.3201966&radius=500&limit=100


<ipython-input-34-85dca10144b7>:27: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues=json_normalize(venues)


,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Allwyn's Bakery,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",43.759840,-79.324719
1,LCBO,"[{'id': '4bf58dd8d48988d186941735', 'name': 'L...",43.757774,-79.314257
2,Shoppers Drug Mart,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",43.760857,-79.324961
3,Petro-Canada,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",43.757950,-79.315187
4,Pizza Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",43.760231,-79.325666
5,TD Canada Trust,"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",43.757440,-79.314838
6,Parkwoods Coin Laundry,"[{'id': '4bf58dd8d48988d1fc941735', 'name': 'L...",43.760386,-79.324894
7,Family Food Fair,"[{'id': '52f2ab2ebcbc57f1066b8b56', 'name': 'A...",43.760422,-79.325012
8,Parkwoods Village Centre,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",43.760735,-79.324873
9,Dollarama,"[{'id': '52dea92d3cf9994f4e043dbb', 'name': 'D...",43.760341,-79.325519


In [35]:
# define a function to extract the category of venues

def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
       
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
    
# filter the category of each row
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type,axis=1)
#print(nearby_venues)

# clean the columns
nearby_venues.columns=[col.split('.')[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,LCBO,Liquor Store,43.757774,-79.314257
2,Shoppers Drug Mart,Pharmacy,43.760857,-79.324961
3,Petro-Canada,Gas Station,43.757950,-79.315187
4,Pizza Pizza,Pizza Place,43.760231,-79.325666
5,TD Canada Trust,Bank,43.757440,-79.314838
6,Parkwoods Coin Laundry,Laundry Service,43.760386,-79.324894
7,Family Food Fair,ATM,43.760422,-79.325012
8,Parkwoods Village Centre,Shopping Mall,43.760735,-79.324873
9,Dollarama,Discount Store,43.760341,-79.325519


### 3.2.3 explore Toronto neighborhoods by FourSquare API

define a function for each neighborhood

In [40]:
def explore_nearby_venues(names,latitudes,longitudes,radius=500):
    limit=5000
    venue_lists=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            limit)
        
        results=requests.get(url).json()
        try:
            venues=results['response']['groups'][0]['items']
        
            for venues_1 in venues:
                venue=venues_1['venue']['name']
                category=venues_1['venue']['categories']
                latitude=venues_1['venue']['location']['lat']
                longitude=venues_1['venue']['location']['lng']
                venue_lists.append([name,lat,lng,venue,category,latitude,longitude])
        except:
            pass
        
    nearby_venues=pd.DataFrame(venue_lists, columns=['neighborhood','n_lat','n_lng','venue','category','venue_lat','venue_lng'])
    return(nearby_venues)

exporing the venues for each neighborhood in Toronto

In [41]:

toronto_venues=explore_nearby_venues(df_final['Neighborhood'],df_final['Latitude'],df_final['Longitude'])

toronto_venues.head()

,neighborhood,n_lat,n_lng,venue,category,venue_lat,venue_lng
0,Parkwoods,43.7588,-79.320197,Allwyn's Bakery,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",43.759840,-79.324719
1,Parkwoods,43.7588,-79.320197,LCBO,"[{'id': '4bf58dd8d48988d186941735', 'name': 'L...",43.757774,-79.314257
2,Parkwoods,43.7588,-79.320197,Shoppers Drug Mart,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",43.760857,-79.324961
3,Parkwoods,43.7588,-79.320197,Petro-Canada,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",43.757950,-79.315187
4,Parkwoods,43.7588,-79.320197,Pizza Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",43.760231,-79.325666


In [42]:
toronto_venues.shape

(5353, 7)

get the category of the venue

In [43]:
def get_category_type_1(row):
    try:
        categories_list=row['category']
    except:
        categories_list=row['venue.categories']
       
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
    
toronto_venues['category']=toronto_venues.apply(get_category_type_1,axis=1)
toronto_venues.head()

,neighborhood,n_lat,n_lng,venue,category,venue_lat,venue_lng
0,Parkwoods,43.7588,-79.320197,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Parkwoods,43.7588,-79.320197,LCBO,Liquor Store,43.757774,-79.314257
2,Parkwoods,43.7588,-79.320197,Shoppers Drug Mart,Pharmacy,43.760857,-79.324961
3,Parkwoods,43.7588,-79.320197,Petro-Canada,Gas Station,43.757950,-79.315187
4,Parkwoods,43.7588,-79.320197,Pizza Pizza,Pizza Place,43.760231,-79.325666


In [44]:
toronto_venues.groupby('neighborhood').count()

,n_lat,n_lng,venue,category,venue_lat,venue_lng
neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt North,28,28,28,28,28,28
Albion Gardens,10,10,10,10,10,10
Bathurst Quay,24,24,24,24,24,24
Beaumonde Heights,1,1,1,1,1,1
...,...,...,...,...,...,...
Willowdale,47,47,47,47,47,47
Woburn,22,22,22,22,22,22
Woodbine Heights,9,9,9,9,9,9


In [45]:
unique_l=toronto_venues['category'].unique()
#print(unique_l)
print('there are {} unique categories.'.format(len(toronto_venues['category'].unique())))

there are 326 unique categories.


### 3.3 analyze each neighborhood in Toronto

data cleaning, tranfor the category/str columns to numeric columns

In [46]:
toronto_onehot=pd.get_dummies(toronto_venues[['category']],prefix="",prefix_sep="")
toronto_onehot['Neighborhood']=toronto_venues['neighborhood']
fixed_columns=[toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot=toronto_onehot[fixed_columns]
#print(toronto_onehot.head())
print(toronto_onehot.shape)

(5353, 326)


In [47]:
toronto_grouped=toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.03,...,0.0,0.01,0.0,0.0,0.000000,0.0,0.0,0.01,0.0,0.0
1,Agincourt North,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,...,0.0,0.00,0.0,0.0,0.035714,0.0,0.0,0.00,0.0,0.0
2,Albion Gardens,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,...,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0
3,Bathurst Quay,0.0,0.0,0.0,0.0,0.0,0.041667,0.041667,0.0,0.00,...,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0
4,Beaumonde Heights,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,...,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0


In [48]:
toronto_grouped.shape

(188, 326)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [92]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    #print(ind)
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
#print(columns)

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
#print(neighborhoods_venues_sorted)

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Gym,Cosmetics Shop,Café,Restaurant,Seafood Restaurant,Japanese Restaurant,Gastropub,Italian Restaurant,American Restaurant
1,Agincourt North,Bank,Bakery,Chinese Restaurant,Restaurant,Clothing Store,Japanese Restaurant,Juice Bar,Movie Theater,Frozen Yogurt Shop,Coffee Shop
2,Albion Gardens,Grocery Store,Beer Store,Pharmacy,Pizza Place,Fast Food Restaurant,Auto Garage,Sandwich Place,Fried Chicken Joint,Caribbean Restaurant,Electronics Store
3,Bathurst Quay,Coffee Shop,Café,Park,Grocery Store,Harbor / Marina,Bank,Ramen Restaurant,Garden,Japanese Restaurant,Diner
4,Beaumonde Heights,Carpet Store,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room


### 3.4 Cluster the neighborhoods

#### 3.4.1 Run k-means to cluster the neighborhood into 5 clusters

In [93]:
kclusters=5
toronto_grouped_cluster=toronto_grouped.drop(columns=['Neighborhood'])
toronto_grouped_cluster.head()

from sklearn.cluster import KMeans

kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(toronto_grouped_cluster)
print(kmeans.labels_[:10])


[3 3 3 3 3 3 3 3 3 3]


In [94]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_final

# merge toronto_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
print(set(toronto_merged['Cluster Labels']))

toronto_merged.dropna(inplace=True)
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype(int)

toronto_merged.head()


{nan, 1.0, 0.0, 3.0, 4.0, nan, nan, 2.0}


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.758800,-79.320197,3,Pharmacy,Liquor Store,Chinese Restaurant,Gas Station,Laundry Service,Discount Store,Coffee Shop,Bank,Caribbean Restaurant,Shopping Mall
1,M4A,North York,Victoria Village,43.732658,-79.311189,3,Thai Restaurant,Spa,Middle Eastern Restaurant,Asian Restaurant,Bus Line,Escape Room,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
2,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,3,Coffee Shop,Restaurant,Thai Restaurant,Grocery Store,Food Truck,Auto Dealership,Pub,Pool,Sushi Restaurant,Electronics Store
3,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150,3,Coffee Shop,Café,Hotel,Restaurant,Pizza Place,Italian Restaurant,Sporting Goods Shop,Sandwich Place,Brewery,Music Venue
4,M6A,North York,Lawrence Manor,43.722079,-79.437507,3,Kids Store,Bank,Doctor's Office,Electronics Store,Park,Escape Room,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant


#### 3.4.2 Visualize the resulting clusters

In [95]:
import numpy as np
map_clusters=folium.Map(location=[latitude, longitude],zoom_start=11)
map_clusters
x=np.arange(kclusters)
print(range(kclusters))
ys=[i+x+(i*x)**2 for i in range(kclusters)]
ys

import matplotlib.cm as cm
import matplotlib.colors as colors
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
colors_array
rainbow=[colors.rgb2hex(i) for i in colors_array]
rainbow

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

range(0, 5)


### 3.5 Examine Clusters

In [96]:
cluster0=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster1=toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster2=toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster3=toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster4=toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [98]:
cluster0

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Etobicoke,0,Café,Women's Store,Flower Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room


In [99]:
cluster1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,North York,1,Metro Station,Playground,Grocery Store,Japanese Restaurant,Asian Restaurant,Bakery,Women's Store,Elementary School,Drugstore,Dumpling Restaurant
24,East York,1,Skating Rink,Pharmacy,ATM,Park,Dance Studio,Athletics & Sports,Bus Stop,Intersection,Escape Room,Drugstore
52,Scarborough,1,Coffee Shop,Pharmacy,Gym,Fast Food Restaurant,Chinese Restaurant,Shopping Mall,Pub,Supermarket,Donut Shop,Drugstore
63,Scarborough,1,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Asian Restaurant,Women's Store,Elementary School,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
65,Scarborough,1,Intersection,Grocery Store,Gas Station,Beer Store,Women's Store,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School
69,East Toronto,1,Vietnamese Restaurant,Chinese Restaurant,Bakery,Bar,Fast Food Restaurant,Cheese Shop,Coffee Shop,Asian Restaurant,Grocery Store,Moving Target
85,North York,1,Convenience Store,American Restaurant,Chinese Restaurant,Coffee Shop,Filipino Restaurant,Field,Drugstore,Dumpling Restaurant,Fish Market,Eastern European Restaurant
91,Scarborough,1,Coffee Shop,Pharmacy,Gym,Fast Food Restaurant,Chinese Restaurant,Shopping Mall,Pub,Supermarket,Donut Shop,Drugstore
112,Scarborough,1,Bowling Alley,Plaza,Fast Food Restaurant,Furniture / Home Store,Beer Store,Electronics Store,Clothing Store,Asian Restaurant,Coffee Shop,Chinese Restaurant
113,Scarborough,1,Middle Eastern Restaurant,Pizza Place,Indian Restaurant,Mediterranean Restaurant,Grocery Store,Fast Food Restaurant,Chinese Restaurant,Supermarket,Dessert Shop,Intersection


In [100]:
cluster2

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
150,Scarborough,2,Playground,Women's Store,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School
167,Etobicoke,2,Playground,Women's Store,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School
175,Scarborough,2,Playground,Women's Store,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School


In [101]:
cluster3

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Pharmacy,Liquor Store,Chinese Restaurant,Gas Station,Laundry Service,Discount Store,Coffee Shop,Bank,Caribbean Restaurant,Shopping Mall
1,North York,3,Thai Restaurant,Spa,Middle Eastern Restaurant,Asian Restaurant,Bus Line,Escape Room,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
2,Downtown Toronto,3,Coffee Shop,Restaurant,Thai Restaurant,Grocery Store,Food Truck,Auto Dealership,Pub,Pool,Sushi Restaurant,Electronics Store
3,Downtown Toronto,3,Coffee Shop,Café,Hotel,Restaurant,Pizza Place,Italian Restaurant,Sporting Goods Shop,Sandwich Place,Brewery,Music Venue
4,North York,3,Kids Store,Bank,Doctor's Office,Electronics Store,Park,Escape Room,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
196,Etobicoke,3,Coffee Shop,Breakfast Spot,Dessert Shop,Pub,Sushi Restaurant,Italian Restaurant,Bank,Pharmacy,Pool Hall,Smoke Shop
198,Etobicoke,3,Restaurant,Sporting Goods Shop,BBQ Joint,Italian Restaurant,Coffee Shop,Yoga Studio,Intersection,Event Space,Liquor Store,Movie Theater
199,Etobicoke,3,Coffee Shop,Café,Restaurant,Boutique,Clothing Store,Italian Restaurant,Spa,Hotel,Park,French Restaurant
200,Etobicoke,3,Coffee Shop,Breakfast Spot,Dessert Shop,Pub,Sushi Restaurant,Italian Restaurant,Bank,Pharmacy,Pool Hall,Smoke Shop


In [102]:
cluster4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,4,Park,Fast Food Restaurant,Caribbean Restaurant,Escape Room,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
21,Scarborough,4,Park,Dongbei Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant
27,Etobicoke,4,Park,Dongbei Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant
30,Etobicoke,4,Piano Bar,Golf Course,Baseball Field,Park,Elementary School,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
38,East York,4,Convenience Store,Sandwich Place,Japanese Restaurant,Park,Women's Store,Escape Room,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
43,North York,4,Convenience Store,Playground,Baseball Field,Park,Women's Store,Escape Room,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
54,North York,4,Women's Store,Park,Intersection,Tennis Court,Escape Room,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
56,North York,4,Park,Middle Eastern Restaurant,Baseball Field,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School
60,Downtown Toronto,4,Park,Harbor / Marina,Music Venue,Café,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School
64,Scarborough,4,Park,Metro Station,Deli / Bodega,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School
